In [1]:
import psycopg2 
import pandas as pd


In [2]:
PGHOST='localhost'
PGDATABASE='postgres'
PGUSER='postgres'
PGPASSWORD='postgres'

In [5]:
# Set up a connection to the postgres server.
conn_string = "host="+ PGHOST +" port="+ "5432" +" dbname="+ PGDATABASE +" user=" + PGUSER \
+" password="+ PGPASSWORD
connection=pg.connect(conn_string)

print("Connected!")


Connected!


In [30]:
df = pd.read_sql("SELECT * FROM h1b_yearly_performance_model ", connection)

In [31]:
df.head()

,employment_duration_years,us_region,annual_salary_band,naics_code_2,yyyy,case_status
0,2,Northeast,b1,52,2012,0
1,2,Northeast,b2,54,2012,1
2,2,South,b1,71,2012,1
3,2,West,b2,45,2012,1
4,2,South,b2,54,2012,1


In [32]:
df1 = pd.get_dummies(df['us_region'], drop_first=True)

In [33]:
df1.head()

,Northeast,South,West
0,1,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,1,0


In [34]:
df = pd.concat([df1,df], axis=1)

In [35]:
df.head()

,Northeast,South,West,employment_duration_years,us_region,annual_salary_band,naics_code_2,yyyy,case_status
0,1,0,0,2,Northeast,b1,52,2012,0
1,1,0,0,2,Northeast,b2,54,2012,1
2,0,1,0,2,South,b1,71,2012,1
3,0,0,1,2,West,b2,45,2012,1
4,0,1,0,2,South,b2,54,2012,1


In [36]:
df.drop('us_region', axis=1, inplace=True)

In [37]:
df.head()

,Northeast,South,West,employment_duration_years,annual_salary_band,naics_code_2,yyyy,case_status
0,1,0,0,2,b1,52,2012,0
1,1,0,0,2,b2,54,2012,1
2,0,1,0,2,b1,71,2012,1
3,0,0,1,2,b2,45,2012,1
4,0,1,0,2,b2,54,2012,1


In [38]:
df2 = pd.get_dummies(df['annual_salary_band'], drop_first=True)

In [39]:
df2.head()

,b2,b3,b4,b5,b6
0,0,0,0,0,0
1,1,0,0,0,0
2,0,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [40]:
df = pd.concat([df2,df], axis=1)

In [41]:
df.head()

,b2,b3,b4,b5,b6,Northeast,South,West,employment_duration_years,annual_salary_band,naics_code_2,yyyy,case_status
0,0,0,0,0,0,1,0,0,2,b1,52,2012,0
1,1,0,0,0,0,1,0,0,2,b2,54,2012,1
2,0,0,0,0,0,0,1,0,2,b1,71,2012,1
3,1,0,0,0,0,0,0,1,2,b2,45,2012,1
4,1,0,0,0,0,0,1,0,2,b2,54,2012,1


In [42]:
df.drop('annual_salary_band', axis=1, inplace=True)

In [43]:
df.drop('yyyy', axis=1, inplace=True)

In [44]:
df.head()

,b2,b3,b4,b5,b6,Northeast,South,West,employment_duration_years,naics_code_2,case_status
0,0,0,0,0,0,1,0,0,2,52,0
1,1,0,0,0,0,1,0,0,2,54,1
2,0,0,0,0,0,0,1,0,2,71,1
3,1,0,0,0,0,0,0,1,2,45,1
4,1,0,0,0,0,0,1,0,2,54,1


In [45]:
X=df.iloc[:,:-1]
y=df.iloc[:,[10]]

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [47]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [48]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9358975860084837


In [50]:
clf.feature_importances_

array([0.0395888 , 0.0363471 , 0.00888499, 0.00334819, 0.26813383,
       0.00968463, 0.01163072, 0.02740965, 0.11257332, 0.48239876])